<a href="https://colab.research.google.com/github/MehrDataPythonist/daily-dev-/blob/main/Welcome_To_Colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##سلول 1: نصب و واردسازی کتابخانه‌های مورد نیاز


In [1]:
!pip install gdown
!pip install ultralytics
!pip install albumentations
!pip install opencv-python-headless

import os
import random
import shutil
from shutil import copyfile
import cv2
import numpy as np
import matplotlib.pyplot as plt
from ultralytics import YOLO

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 793.0/793.0 kB 14.4 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.whl (176.2 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-

##سلول 2: دانلود و استخراج فایل زیپ از Google Drive


In [2]:
import gdown
import zipfile
import os

file_url = 'https://drive.google.com/uc?id=1Bw9lkOC1yNPie2yi7Rajx82pehi8UtVh'
zip_path = 'dataset.zip'
gdown.download(file_url, zip_path, quiet=False)

extract_dir = 'dataset/'
with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extract_dir)

extracted_files = os.listdir(extract_dir)
print(f"تعداد فایل‌های استخراج شده: {len(extracted_files)}")
print("فایل‌های استخراج شده:", extracted_files)

internal_files = []
for root, dirs, files in os.walk(extract_dir):
    for file in files:
        internal_files.append(os.path.join(root, file))

print(f"تعداد فایل‌های داخلی: {len(internal_files)}")
print("فایل‌های داخلی:", internal_files)

if len(internal_files) == 0:
    print("هیچ فایلی در مسیر داده شده وجود ندارد. لطفاً مسیر فایل‌ها را بررسی کنید.")
else:
    print("فایل‌ها به درستی استخراج شدند.")

Downloading...
From (original): https://drive.google.com/uc?id=1Bw9lkOC1yNPie2yi7Rajx82pehi8UtVh
From (redirected): https://drive.google.com/uc?id=1Bw9lkOC1yNPie2yi7Rajx82pehi8UtVh&confirm=t&uuid=a4013275-a3e4-42ed-9648-af439bd397a4
To: /content/dataset.zip
100%|██████████| 64.6M/64.6M [00:00<00:00, 83.0MB/s]


تعداد فایل‌های استخراج شده: 73
فایل‌های استخراج شده: ['062.jpg', '026.jpg', '069.jpg', '005.jpg', '017.jpg', '010.jpg', '041.jpg', '053.jpg', '019.jpg', '025.jpg', '021.jpg', '066.jpg', '004.jpg', '018.jpg', '028.jpg', '007.jpg', '055.jpg', '036.jpg', '048.jpg', '024.jpg', '061.jpg', '031.jpg', '030.jpg', '020.jpg', '013.jpg', '014.jpg', '065.jpg', '011.jpg', '049.jpg', '044.jpg', '000.jpg', '006.jpg', '032.jpg', '039.jpg', '046.jpg', '072.jpg', '015.jpg', '052.jpg', '070.jpg', '003.jpg', '022.jpg', '038.jpg', '060.jpg', '029.jpg', '033.jpg', '057.jpg', '064.jpg', '047.jpg', '001.jpg', '009.jpg', '050.jpg', '023.jpg', '027.jpg', '063.jpg', '008.jpg', '002.jpg', '071.jpg', '037.jpg', '054.jpg', '035.jpg', '012.jpg', '059.jpg', '068.jpg', '058.jpg', '042.jpg', '040.jpg', '051.jpg', '043.jpg', '056.jpg', '034.jpg', '067.jpg', '016.jpg', '045.jpg']
تعداد فایل‌های داخلی: 73
فایل‌های داخلی: ['dataset/062.jpg', 'dataset/026.jpg', 'dataset/069.jpg', 'dataset/005.jpg', 'dataset/017.jpg', 'datas

##سلول 3: برش پلاک‌ها و برچسب‌گذاری تصاویر جدید


In [3]:
def crop_license_plate(image, detections):
    plates = []
    for (x1, y1, x2, y2) in detections:
        plate = image[int(y1):int(y2), int(x1):int(x2)]
        plates.append((plate, (x1, y1, x2, y2)))
    return plates

model = YOLO('yolov5s.pt')

images_dir = 'dataset/'
images = [img for img in os.listdir(images_dir) if img.endswith('.jpg')]

if len(images) == 0:
    print("هیچ تصویری در مسیر داده شده وجود ندارد. لطفاً مسیر تصاویر را بررسی کنید.")
else:
    for img in images:
        image_path = os.path.join(images_dir, img)
        image = cv2.imread(image_path)
        results = model(image_path)

        for result in results:
            boxes = result.boxes.xyxy.cpu().numpy()
            plates = crop_license_plate(image, boxes)

            for i, (plate, box) in enumerate(plates):
                plate_path = os.path.join(images_dir, f"{os.path.splitext(img)[0]}_plate_{i}.jpg")
                cv2.imwrite(plate_path, plate)

                label_path = os.path.join(images_dir, f"{os.path.splitext(img)[0]}_plate_{i}.txt")
                with open(label_path, 'w') as f:
                    class_id = 0
                    x_center = (box[0] + box[2]) / 2 / image.shape[1]
                    y_center = (box[1] + box[3]) / 2 / image.shape[0]
                    width = (box[2] - box[0]) / image.shape[1]
                    height = (box[3] - box[1]) / image.shape[0]
                    f.write(f"{class_id} {x_center} {y_center} {width} {height}\n")

    print("پلاک‌ها با موفقیت برش داده شدند و فایل‌های برچسب ایجاد شدند.")

PRO TIP 💡 Replace 'model=yolov5s.pt' with new 'model=yolov5su.pt'.
YOLOv5 'u' models are trained with https://github.com/ultralytics/ultralytics and feature improved performance vs standard YOLOv5 models trained with https://github.com/ultralytics/yolov5.



100%|██████████| 17.7M/17.7M [00:00<00:00, 368MB/s]



image 1/1 /content/dataset/062.jpg: 384x640 (no detections), 426.4ms
Speed: 15.6ms preprocess, 426.4ms inference, 14.7ms postprocess per image at shape (1, 3, 384, 640)

image 1/1 /content/dataset/026.jpg: 384x640 (no detections), 285.7ms
Speed: 1.4ms preprocess, 285.7ms inference, 0.7ms postprocess per image at shape (1, 3, 384, 640)

image 1/1 /content/dataset/069.jpg: 384x640 1 car, 286.8ms
Speed: 3.1ms preprocess, 286.8ms inference, 12.4ms postprocess per image at shape (1, 3, 384, 640)

image 1/1 /content/dataset/005.jpg: 448x640 1 person, 1 car, 389.6ms
Speed: 2.6ms preprocess, 389.6ms inference, 1.4ms postprocess per image at shape (1, 3, 448, 640)

image 1/1 /content/dataset/017.jpg: 512x640 1 truck, 630.3ms
Speed: 3.9ms preprocess, 630.3ms inference, 1.6ms postprocess per image at shape (1, 3, 512, 640)

image 1/1 /content/dataset/010.jpg: 448x640 1 person, 6 cars, 1 bus, 519.0ms
Speed: 3.7ms preprocess, 519.0ms inference, 5.0ms postprocess per image at shape (1, 3, 448, 640)



##سلول 5: تقسیم داده‌ها به Training و Validation




In [4]:
train_dir = os.path.join('dataset', 'train_sample')
val_dir = os.path.join('dataset', 'val_sample')

for dir in [train_dir, val_dir]:
    os.makedirs(os.path.join(dir, 'images'), exist_ok=True)
    os.makedirs(os.path.join(dir, 'labels'), exist_ok=True)

all_images = [img for img in os.listdir(images_dir) if img.endswith('.jpg')]
random.shuffle(all_images)

train_size = int(0.8 * len(all_images))

train_images = all_images[:train_size]
val_images = all_images[train_size:]

def copy_files(file_list, source_img_dir, target_img_dir, source_lbl_dir, target_lbl_dir):
    for file in file_list:
        img_src = os.path.join(source_img_dir, file)
        lbl_src = os.path.join(source_lbl_dir, file.replace('.jpg', '.txt'))
        img_dst = os.path.join(target_img_dir, file)
        lbl_dst = os.path.join(target_lbl_dir, file.replace('.jpg', '.txt'))
        if os.path.exists(lbl_src) and img_src != img_dst and lbl_src != lbl_dst:
            copyfile(img_src, img_dst)
            copyfile(lbl_src, lbl_dst)

copy_files(train_images, images_dir, os.path.join(train_dir, 'images'), images_dir, os.path.join(train_dir, 'labels'))
copy_files(val_images, images_dir, os.path.join(val_dir, 'images'), images_dir, os.path.join(val_dir, 'labels'))

print(f"تعداد تصاویر آموزش: {len(train_images)}")
print(f"تعداد تصاویر اعتبارسنجی: {len(val_images)}")

تعداد تصاویر آموزش: 184
تعداد تصاویر اعتبارسنجی: 46


##سلول 7: آماده‌سازی فایل پیکربندی YAML


In [5]:
yaml_content = """
path: /content/dataset
train: /content/dataset/train_sample/images
val: /content/dataset/val_sample/images

nc: 1
names: ['plate']
"""

yaml_path = os.path.join('/content/dataset', 'dataset_sample.yaml')
with open(yaml_path, 'w') as f:
    f.write(yaml_content)

print("فایل پیکربندی YAML با موفقیت ایجاد شد.")

فایل پیکربندی YAML با موفقیت ایجاد شد.


##سلول 8: آموزش مدل YOLOv5


In [ ]:
model = YOLO('yolov5su.pt')

results = model.train(
    data=yaml_path,
    epochs=5,
    imgsz=640,
    batch=4,
    name='digit_detection_sample_small'
)

print("آموزش مدل با موفقیت به پایان رسید.")

Ultralytics YOLOv8.2.41 🚀 Python-3.10.12 torch-2.3.0+cu121 CPU (Intel Xeon 2.20GHz)
engine/trainer: task=detect, mode=train, model=yolov5su.pt, data=/content/dataset/dataset_sample.yaml, epochs=5, time=None, patience=100, batch=4, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=digit_detection_sample_small, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=False, show_label

100%|██████████| 755k/755k [00:00<00:00, 83.1MB/s]


Overriding model.yaml nc=80 with nc=1

                   from  n    params  module                                       arguments                     
  0                  -1  1      3520  ultralytics.nn.modules.conv.Conv             [3, 32, 6, 2, 2]              
  1                  -1  1     18560  ultralytics.nn.modules.conv.Conv             [32, 64, 3, 2]                
  2                  -1  1     18816  ultralytics.nn.modules.block.C3              [64, 64, 1]                   
  3                  -1  1     73984  ultralytics.nn.modules.conv.Conv             [64, 128, 3, 2]               
  4                  -1  2    115712  ultralytics.nn.modules.block.C3              [128, 128, 2]                 
  5                  -1  1    295424  ultralytics.nn.modules.conv.Conv             [128, 256, 3, 2]              
  6                  -1  3    625152  ultralytics.nn.modules.block.C3              [256, 256, 3]                 
  7                  -1  1   1180672  ultralytics

train: Scanning /content/dataset/train_sample/labels... 127 images, 0 backgrounds, 1 corrupt: 100%|██████████| 128/128 [00:00<00:00, 1664.45it/s]

train: WARNING ⚠️ /content/dataset/train_sample/images/042_plate_3.jpg: ignoring corrupt image/label: image size (19, 7) <10 pixels
train: New cache created: /content/dataset/train_sample/labels.cache


...


      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


        4/5         0G      2.149      2.992      2.193          6        640: 100%|██████████| 32/32 [04:16<00:00,  8.01s/it]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 4/4 [00:15<00:00,  3.78s/it]

                   all         28         28    0.00703      0.143     0.0062    0.00146



      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


        5/5         0G      2.096      2.933      2.137         12        640:  75%|███████▌  | 24/32 [03:10<01:01,  7.70s/it]

##سلول 9: ارزیابی مدل و نمایش پلات‌های مختلف


In [ ]:
model = YOLO('runs/detect/digit_detection_sample_small/weights/best.pt')

metrics = model.val(data=yaml_path)

print("نتایج ارزیابی مدل:")
print(metrics)

results_dir = 'runs/detect/digit_detection_sample_small'

import matplotlib.pyplot as plt
import matplotlib.image as mpimg

def display_image(image_path, title=''):
    img = mpimg.imread(image_path)
    plt.figure(figsize=(10, 10))
    plt.imshow(img)
    plt.title(title)
    plt.axis('off')
    plt.show()

if os.path.exists(os.path.join(results_dir, 'val_batch0_labels.jpg')):
    display_image(os.path.join(results_dir, 'val_batch0_labels.jpg'), 'Validation Batch 0 Labels')
if os.path.exists(os.path.join(results_dir, 'val_batch0_pred.jpg')):
    display_image(os.path.join(results_dir, 'val_batch0_pred.jpg'), 'Validation Batch 0 Predictions')
if os.path.exists(os.path.join(results_dir, 'labels.jpg')):
    display_image(os.path.join(results_dir, 'labels.jpg'), 'Labels')
if os.path.exists(os.path.join(results_dir, 'confusion_matrix.png')):
    display_image(os.path.join(results_dir, 'confusion_matrix.png'), 'Confusion Matrix')
if os.path.exists(os.path.join(results_dir, 'PR_curve.png')):
    display_image(os.path.join(results_dir, 'PR_curve.png'), 'Precision-Recall Curve')
if os.path.exists(os.path.join(results_dir, 'F1_curve.png')):
    display_image(os.path.join(results_dir, 'F1 Curve'))
if os.path.exists(os.path.join(results_dir, 'P_curve.png')):
    display_image(os.path.join(results_dir, 'Precision Curve'))
if os.path.exists(os.path.join(results_dir, 'R_curve.png')):
    display_image(os.path.join(results_dir, 'Recall Curve'))